In [10]:
!pip install pandas altair

import pandas as pd

import altair as alt

# Ustawienie, aby pandas wyświetlał wszystkie kolumny
pd.set_option('display.max_columns', None)


file_path = 'sales_data_sample.csv'


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
df = pd.read_csv(file_path, encoding='latin1')

In [13]:
print("\nPierwsze 5 wierszy (head):")
print(df.head())


Pierwsze 5 wierszy (head):
   ORDERNUMBER  QUANTITYORDERED  PRICEEACH  ORDERLINENUMBER    SALES  \
0        10107               30      95.70                2  2871.00   
1        10121               34      81.35                5  2765.90   
2        10134               41      94.74                2  3884.34   
3        10145               45      83.26                6  3746.70   
4        10159               49     100.00               14  5205.27   

         ORDERDATE   STATUS  QTR_ID  MONTH_ID  YEAR_ID  PRODUCTLINE  MSRP  \
0   2/24/2003 0:00  Shipped       1         2     2003  Motorcycles    95   
1    5/7/2003 0:00  Shipped       2         5     2003  Motorcycles    95   
2    7/1/2003 0:00  Shipped       3         7     2003  Motorcycles    95   
3   8/25/2003 0:00  Shipped       3         8     2003  Motorcycles    95   
4  10/10/2003 0:00  Shipped       4        10     2003  Motorcycles    95   

  PRODUCTCODE              CUSTOMERNAME             PHONE  \
0    S10_1678  

In [15]:
print("\nInformacje o danych (info):")
df.info()


Informacje o danych (info):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2823 entries, 0 to 2822
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ORDERNUMBER       2823 non-null   int64  
 1   QUANTITYORDERED   2823 non-null   int64  
 2   PRICEEACH         2823 non-null   float64
 3   ORDERLINENUMBER   2823 non-null   int64  
 4   SALES             2823 non-null   float64
 5   ORDERDATE         2823 non-null   object 
 6   STATUS            2823 non-null   object 
 7   QTR_ID            2823 non-null   int64  
 8   MONTH_ID          2823 non-null   int64  
 9   YEAR_ID           2823 non-null   int64  
 10  PRODUCTLINE       2823 non-null   object 
 11  MSRP              2823 non-null   int64  
 12  PRODUCTCODE       2823 non-null   object 
 13  CUSTOMERNAME      2823 non-null   object 
 14  PHONE             2823 non-null   object 
 15  ADDRESSLINE1      2823 non-null   object 
 16  ADDRESSLINE2 

In [16]:
#CZYSZCZENIE DANYCH

In [17]:
df['ORDERDATE'] = pd.to_datetime(df['ORDERDATE'])

In [18]:
columns_to_drop = [
    'ADDRESSLINE2', 
    'STATE',        
    'POSTALCODE',   
    'TERRITORY',    
    'PHONE',        
    'CONTACTLASTNAME', 
    'CONTACTFIRSTNAME' 
]

In [19]:
df.drop(columns=columns_to_drop, inplace=True)
print(f"Usunięto {len(columns_to_drop)} zbędnych kolumn.")

Usunięto 7 zbędnych kolumn.


In [20]:
print("\nInformacje o danych (info) PO CZYSZCZENIU:")
df.info()

print("\nPróbka danych (head) po czyszczeniu:")
print(df.head())


Informacje o danych (info) PO CZYSZCZENIU:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2823 entries, 0 to 2822
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   ORDERNUMBER      2823 non-null   int64         
 1   QUANTITYORDERED  2823 non-null   int64         
 2   PRICEEACH        2823 non-null   float64       
 3   ORDERLINENUMBER  2823 non-null   int64         
 4   SALES            2823 non-null   float64       
 5   ORDERDATE        2823 non-null   datetime64[ns]
 6   STATUS           2823 non-null   object        
 7   QTR_ID           2823 non-null   int64         
 8   MONTH_ID         2823 non-null   int64         
 9   YEAR_ID          2823 non-null   int64         
 10  PRODUCTLINE      2823 non-null   object        
 11  MSRP             2823 non-null   int64         
 12  PRODUCTCODE      2823 non-null   object        
 13  CUSTOMERNAME     2823 non-null   object        
 

In [21]:
#Analiza Trendów Sprzedaży

In [22]:
#Używamy .dt.to_period('M'), aby dostać format 'YYYY-MM'
df['YEAR_MONTH'] = df['ORDERDATE'].dt.to_period('M')

In [24]:
df['MONTH'] = df['ORDERDATE'].dt.month_name()

In [25]:
#Agregacja danych do analizy trendu (Sprzedaż w czasie)
sales_trend = df.groupby('YEAR_MONTH')['SALES'].sum().reset_index()
sales_trend['YEAR_MONTH'] = sales_trend['YEAR_MONTH'].dt.to_timestamp()

In [26]:
#Agregacja danych do analizy sezonowości (Sprzedaż wg miesięcy)
sales_seasonal = df.groupby('MONTH')['SALES'].sum().reset_index()

In [28]:
#Definiowanie poprawnej kolejności miesięcy do sortowania wykresu
month_order = [
    'January', 'February', 'March', 'April', 'May', 'June',
    'July', 'August', 'September', 'October', 'November', 'December'
]

In [29]:
#w celu unkinięcia błędu maxrowserror
alt.data_transformers.enable('json')


DataTransformerRegistry.enable('json')

In [32]:
#wykres 1 trend sprzedaży w czasie liniowy
chart_trend= alt.Chart(sales_trend).mark_line(point=True).encode(
    x=alt.X('YEAR_MONTH', title='Data (rok-miesiąc)'),
            y=alt.Y('SALES', title='Całkowita sprzedaż w USD'),
            tooltip=[
        alt.Tooltip('YEAR_MONTH', title='Miesiąc', format='%Y-%m'),
        alt.Tooltip('SALES', title='Sprzedaż', format=',.2f')
    ]
).properties(
    title='Trend sprzedaży w czasie (2003-2005)',
    width=800,
    height=400
).interactive()

In [33]:
#wykres 2 sezonowość sprzedaży słupkowy
chart_seasonal = alt.Chart(sales_seasonal).mark_bar().encode(
    x=alt.X('MONTH', title='Miesiąc', sort=month_order),
    y=alt.Y('SALES', title='Całkowita Sprzedaż (USD)'),
    tooltip=[
        alt.Tooltip('MONTH', title='Miesiąc'),
        alt.Tooltip('SALES', title='Sprzedaż', format=',.0f')
    ]
).properties(
    title='Sezonowość sprzedaży (Suma ze wszystkich lat)',
    width=800
).interactive()

In [34]:
chart_trend.save('sales_trend_over_time.html')
chart_seasonal.save('seasonal_sales_chart.html')

In [35]:
#analiza produktów


In [38]:
#Analiza Linii Produktowych (Przychód),grupujemy po 'PRODUCTLINE' i sumujemy sprzedaż

In [39]:
productline_sales=df.groupby('PRODUCTLINE')['SALES'].sum().reset_index()

In [41]:
print("\nCałkowita sprzedaż wg Linii Produktowej:")
print(productline_sales.sort_values(by='SALES', ascending=False))


Całkowita sprzedaż wg Linii Produktowej:
        PRODUCTLINE       SALES
0      Classic Cars  3919615.66
6      Vintage Cars  1903150.84
1       Motorcycles  1166388.34
5  Trucks and Buses  1127789.84
2            Planes   975003.57
3             Ships   714437.13
4            Trains   226243.47


In [43]:
#analiza bessellerów liczba zamówień

In [44]:
bestselling_products=df['PRODUCTCODE'].value_counts().reset_index()

In [45]:
bestselling_products.columns=['PRODUCTCODE', 'NUMBER_OF_ORDERS']

In [46]:
print("\n 10 Najczęściej zamawianych produktów (Bestsellery):")
print(bestselling_products.head(10))


 10 Najczęściej zamawianych produktów (Bestsellery):
  PRODUCTCODE  NUMBER_OF_ORDERS
0    S18_3232                52
1    S12_1666                28
2    S10_4962                28
3    S18_1097                28
4    S10_1949                28
5    S18_2432                28
6    S32_2509                28
7    S24_2840                28
8    S24_1444                28
9    S50_1392                28


In [47]:
#wykres linnie produktowe słupkowy


In [76]:
chart_productline=alt.Chart(productline_sales).mark_bar().encode(
    x=alt.X('PRODUCTLINE', title='Linia produktowa', sort='-y'),
    y=alt.Y('SALES', title='Całkowita sprzedaż w USD'),
    tooltip=[
        alt.Tooltip('PRODUCTLINE', title='Linia produktowa'),
        alt.Tooltip('SALES', title='Sprzedaż', format=',.0f')]
).properties(title='Całkowita sprzedaż wg. Linii Produktowej',    width=800
).interactive()


In [77]:
chart_productline.save('productline_sales.html')

In [74]:
#wykres top 10 bestsellerów słupkowy
chart_bestsellers=alt.Chart(bestselling_products.head(10)).mark_bar().encode(
    x=alt.X('PRODUCTCODE', title='Kod Produktu', sort='-y'),
    y=alt.Y('NUMBER_OF_ORDERS', title='Liczba Zamówień'),
    tooltip=['PRODUCTCODE', 'NUMBER_OF_ORDERS']
).properties(title='Top 10 Bestsellerów (wg liczby zamówień)',width=800
).interactive()

In [75]:
chart_bestsellers.save('top_10_bestsellers.html')

In [55]:
#pytanie komu sprzedajemy?
#analiza klientów

In [56]:
#Grupujemy po 'CUSTOMERNAME' i sumujemy sprzedaż
customer_sales = df.groupby('CUSTOMERNAME')['SALES'].sum().reset_index()

In [57]:
#sortowanie
top_customers = customer_sales.sort_values(by='SALES', ascending=False).head(10)

In [58]:
print(top_customers)

                    CUSTOMERNAME      SALES
33         Euro Shopping Channel  912294.11
55  Mini Gifts Distributors Ltd.  654858.06
6     Australian Collectors, Co.  200995.41
58            Muscle Machine Inc  197736.94
44             La Rochelle Gifts  180124.90
31       Dragon Souveniers, Ltd.  172989.68
45             Land of Toys Inc.  164069.44
81     The Sharp Gifts Warehouse  160010.27
0                 AV Stores, Co.  157807.81
3        Anna's Decorations, Ltd  153996.13


In [63]:
#wykres
chart_top_customers=alt.Chart(top_customers).mark_bar().encode(
    x=alt.X('CUSTOMERNAME', title='Nazwa klienta', sort='-y'),
    y=alt.Y('SALES', title='Całkowita sprzedaż USD'),
    tooltip=[
        alt.Tooltip('CUSTOMERNAME', title='Nazwa Klienta'),
        alt.Tooltip('SALES', title='Sprzedaż', format=',.0f')
    ]
).properties(
    title='Top 10 Klientów (wg całkowitej sprzedaży)',
    width=800
).interactive()

In [64]:
chart_top_customers.save('top_10_customers.html')

In [65]:
#analiza geograficzna


In [66]:
#grupujemy po 'COUNTRY' i sum sprzedaż

In [67]:
country_sales = df.groupby('COUNTRY')['SALES'].sum().reset_index()

In [68]:
top_countries = country_sales.sort_values(by='SALES', ascending=False).head(10)

In [69]:
print(top_countries)

      COUNTRY       SALES
18        USA  3627982.83
14      Spain  1215686.92
6      France  1110916.52
0   Australia   630623.10
17         UK   478880.46
9       Italy   374674.31
5     Finland   329581.91
11     Norway   307463.70
13  Singapore   288488.41
4     Denmark   245637.15


In [72]:
chart_top_countries = alt.Chart(top_countries).mark_bar().encode(
    #  słupki od największego
    x=alt.X('COUNTRY', title='Kraj', sort='-y'),
    y=alt.Y('SALES', title='Całkowita Sprzedaż (USD)'),
    tooltip=[
        alt.Tooltip('COUNTRY', title='Kraj'),
        alt.Tooltip('SALES', title='Sprzedaż', format=',.0f')
    ]
).properties(
    title='Top 10 Krajów (wg całkowitej sprzedaży)',
    width=800
).interactive()

In [73]:
chart_top_countries.save('top_10_countries.html')